In [161]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


In [251]:
sphynx.stop()

Stopping Spark session...


/usr/local/spark/python/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning


Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [164]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

# user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
# user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

In [91]:
def get_user_info(start_date, end_date):
    user = load_data_mart("pc", end_date, end_date, "user_master").where(col('lastlogindate') >= start_date).withColumnRenamed("accountid", "account_id")
    user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
    user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")
    membership = load_membership("pubg", "pc", end_date, end_date)
    user = user.join(membership, "account_id")
#     user = user.select("account_id", "membership", "pubg_region").distinct()
    return user

In [102]:
# nov_user = get_user_info("2021-11-01", "2021-11-30")
# dec_user = get_user_info("2021-12-01", "2021-12-31")
feb_user = get_user_info("2022-02-01", "2022-02-28")
march_user = get_user_info("2022-03-01", "2022-03-31")

In [103]:
feb_cash = load_data_mart("pc", "2022-02-01", "2022-02-28", "cash_mtx")
march_cash = load_data_mart("pc", "2022-03-01", "2022-03-31", "cash_mtx")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42631)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


In [104]:
feb_cash = feb_cash.join(feb_user, "account_id", "left")
march_cash = march_cash.join(march_user, "account_id", "left")

In [40]:
feb_user.groupBy("membership").agg(countDistinct("account_id").alias("au")).show()
march_user.groupBy("membership").agg(countDistinct("account_id").alias("au")).show()

+----------+-------+
|membership|     au|
+----------+-------+
|      plus| 111541|
|     basic|6748746|
|    legacy|6401947|
+----------+-------+

+----------+-------+
|membership|     au|
+----------+-------+
|      plus| 144729|
|     basic|6319167|
|    legacy|5797957|
+----------+-------+



In [44]:
feb_cash.groupBy("membership").agg(sum("ingame_revenue").alias("revenue"), countDistinct("account_id").alias("pu")).show()
march_cash.groupBy("membership").agg(sum("ingame_revenue").alias("revenue"), countDistinct("account_id").alias("pu")).show()

+----------+-----------+------+
|membership|    revenue|    pu|
+----------+-----------+------+
|      plus| 3697742.25| 76372|
|     basic| 1027688.73|131777|
|    legacy|17792799.09|623633|
+----------+-----------+------+

+----------+-----------+------+
|membership|    revenue|    pu|
+----------+-----------+------+
|      null|       1.99|     1|
|      plus| 3053227.30| 81644|
|     basic| 1039482.03| 93255|
|    legacy|18996423.49|613567|
+----------+-----------+------+



In [46]:
feb_cash.where(col("product_id") != "gcoinbundledesc.1015").groupBy("membership").agg(sum("ingame_revenue").alias("revenue"), countDistinct("account_id").alias("pu")).show()
march_cash.where(col("product_id") != "gcoinbundledesc.1015").groupBy("membership").agg(sum("ingame_revenue").alias("revenue"), countDistinct("account_id").alias("pu")).show()

+----------+-----------+------+
|membership|    revenue|    pu|
+----------+-----------+------+
|      plus| 2848378.06| 45361|
|     basic| 1027637.01|131774|
|    legacy|17788591.13|623396|
+----------+-----------+------+

+----------+-----------+------+
|membership|    revenue|    pu|
+----------+-----------+------+
|      null|       1.99|     1|
|      plus| 2299121.72| 53079|
|     basic| 1039469.12| 93254|
|    legacy|18992851.40|613360|
+----------+-----------+------+



In [110]:
feb_cash.select("account_id", "membership", "product_id", "product_name", "unit_sold", "ingame_revenue")

DataFrame[account_id: string, membership: string, product_id: string, product_name: string, unit_sold: bigint, ingame_revenue: decimal(38,2)]

In [114]:
topup = load_data_mart("pc", "2021-12-01", "2021-12-01", "gcoin_topup")

In [120]:
total_cash = feb_cash.select("account_id", "membership", "product_id", "product_name", "unit_sold", "ingame_revenue").withColumn("month", lit("feb")).unionByName(march_cash.select("account_id", "membership", "product_id", "product_name", "unit_sold", "ingame_revenue").withColumn("month", lit("march")))
total_cash.where((col("product_id") != "gcoinbundledesc.1015") & (col("membership") == "legacy")).groupBy("month", "product_name").agg(countDistinct("account_id").alias("pu"), sum("unit_sold").alias("unit_sold"), sum("ingame_revenue").alias("revenue")).groupBy("product_name").agg(mean("pu").alias("avg_pu"), mean("unit_sold").alias("avg_unit_sold"), mean("revenue").alias("avg_revenue")).orderBy("avg_unit_sold", ascending=False).toPandas()

,product_name,avg_pu,avg_unit_sold,avg_revenue
0,100 G-Coin,147823.0,334206.0,331029.770000
1,STARTER PACKAGE,240879.0,240879.0,478475.410000
2,1000(+50 Bonus) G-Coin,171218.0,237056.5,2368361.690000
3,HARDCORE HITMAN VALUE PACK,235916.0,235916.0,2352844.800000
4,500(+10 Bonus) G-Coin,145011.0,185761.0,927039.655000
5,2500(+200 Bonus) G-Coin,89583.5,133572.0,3338038.070000
6,10000(+1200 Bonus) G-Coin,26149.5,67771.0,6776463.455000
7,5000(+500 Bonus) G-Coin,41735.5,59904.0,2994656.050000
8,GILT ELEGANCE - 1050 G-COIN SKIN PACK,23.5,23.5,234.765000


In [122]:
total_cash.where((col("product_id") != "gcoinbundledesc.1015") & (col("membership") == "plus")).groupBy("month", "product_name").agg(countDistinct("account_id").alias("pu"), sum("unit_sold").alias("unit_sold"), sum("ingame_revenue").alias("revenue")).groupBy("product_name").agg(mean("pu").alias("avg_pu"), mean("unit_sold").alias("avg_unit_sold"), mean("revenue").alias("avg_revenue")).orderBy("avg_unit_sold", ascending=False).toPandas()

,product_name,avg_pu,avg_unit_sold,avg_revenue
0,100 G-Coin,13622.5,36502.5,36169.910000
1,STARTER PACKAGE,25098.0,25098.0,49767.380000
2,1000(+50 Bonus) G-Coin,15720.0,24459.5,244379.680000
3,500(+10 Bonus) G-Coin,15290.0,22224.5,110919.285000
4,HARDCORE HITMAN VALUE PACK,17475.0,17475.0,173987.850000
5,2500(+200 Bonus) G-Coin,9312.5,14920.5,372878.300000
6,10000(+1200 Bonus) G-Coin,2887.0,13721.0,1371968.040000
7,5000(+500 Bonus) G-Coin,3730.5,6014.5,300673.370000


In [121]:
total_cash.where((col("product_id") != "gcoinbundledesc.1015") & (col("membership") == "basic")).groupBy("month", "product_name").agg(countDistinct("account_id").alias("pu"), sum("unit_sold").alias("unit_sold"), sum("ingame_revenue").alias("revenue")).groupBy("product_name").agg(mean("pu").alias("avg_pu"), mean("unit_sold").alias("avg_unit_sold"), mean("revenue").alias("avg_revenue")).orderBy("avg_unit_sold", ascending=False).toPandas()

,product_name,avg_pu,avg_unit_sold,avg_revenue
0,STARTER PACKAGE,80583.5,80583.5,159915.085000
1,100 G-Coin,31356.5,67437.0,66816.695000
2,500(+10 Bonus) G-Coin,20363.5,24946.5,124512.545000
3,1000(+50 Bonus) G-Coin,17056.5,21723.0,217057.495000
4,HARDCORE HITMAN VALUE PACK,8863.0,8863.0,88024.810000
5,2500(+200 Bonus) G-Coin,6810.5,8565.0,214055.675000
6,5000(+500 Bonus) G-Coin,1624.0,1992.0,99586.835000
7,10000(+1200 Bonus) G-Coin,504.5,1076.0,107591.335000
8,GILT ELEGANCE - 1050 G-COIN SKIN PACK,1.0,1.0,9.990000


In [50]:
feb_cash.where((col("product_id") != "gcoinbundledesc.1015") & (col("membership") == "plus")).groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("unit_sold").alias("unit_sold"), sum("ingame_revenue").alias("revenue")).toPandas()

,product_name,pu,unit_sold,revenue
0,500(+10 Bonus) G-Coin,14883,22043,110011.39
1,STARTER PACKAGE,27575,27575,54684.41
2,1000(+50 Bonus) G-Coin,12688,20032,200141.85
3,100 G-Coin,13900,37052,36710.87
4,5000(+500 Bonus) G-Coin,2896,4389,219412.03
5,2500(+200 Bonus) G-Coin,8218,13353,333702.77
6,10000(+1200 Bonus) G-Coin,3321,18939,1893714.74


In [54]:
march_cash.where((col("product_id") != "gcoinbundledesc.1015") & (col("membership") == "plus")).groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("unit_sold").alias("unit_sold"), sum("ingame_revenue").alias("revenue")).orderBy("pu", ascending=False).toPandas()

,product_name,pu,unit_sold,revenue
0,STARTER PACKAGE,22621,22621,44850.35
1,1000(+50 Bonus) G-Coin,18752,28887,288617.51
2,HARDCORE HITMAN VALUE PACK,17475,17475,173987.85
3,500(+10 Bonus) G-Coin,15697,22406,111827.18
4,100 G-Coin,13345,35953,35628.95
5,2500(+200 Bonus) G-Coin,10407,16488,412053.83
6,5000(+500 Bonus) G-Coin,4565,7640,381934.71
7,10000(+1200 Bonus) G-Coin,2453,8503,850221.34


In [20]:
cash_df = load_data_mart("pc", "2022-01-12", "2022-01-18", "cash_mtx")

In [22]:
membership = load_membership("pubg", "pc", "2022-01-18", "2022-01-18").select("account_id", "membership").distinct()

In [23]:
cash_df = cash_df.join(membership, "account_id")

In [27]:
first_buy_time = cash_df.where(col("membership")!="legacy").groupBy("account_id").agg(min("time").alias("first_buy_time"))
first_buy_item = cash_df.join(first_buy_time, (cash_df.account_id == first_buy_time.account_id) & (cash_df.time == first_buy_time.first_buy_time))

In [29]:
first_buy_item.groupBy("product_name").agg(count("*")).toPandas()

,product_name,count(1)
0,500(+10 Bonus) G-Coin,2955
1,STARTER PACKAGE,98192
2,1000(+50 Bonus) G-Coin,3413
3,100 G-Coin,2621
4,BATTLEGROUNDS PLUS,20493
5,5000(+500 Bonus) G-Coin,238
6,2500(+200 Bonus) G-Coin,1887
7,10000(+1200 Bonus) G-Coin,84
8,GILT ELEGANCE - 1050 G-COIN SKIN PACK,4


In [6]:
# starter package 구매자들 중 legacy 가 아닌 유저들의 현금 구매 상품

cash = pd.read_csv('./data/starter_buyer_cash_buy.csv')

In [13]:
cash = cash[cash.membership != "legacy"]

In [14]:
cash.groupby(["account_id"], as_index=False)[["ingame_revenue", "unit_sold"]].sum().describe()

,ingame_revenue,unit_sold
count,107082.000000,107082.000000
mean,6.405848,1.558198
std,26.099224,1.279308
min,1.910000,1.000000
25%,1.990000,1.000000
50%,1.990000,1.000000
75%,2.980000,2.000000
max,1369.830000,48.000000


In [15]:
cash.groupby(["account_id", "membership"], as_index=False)[["ingame_revenue", "unit_sold"]].sum().groupby("membership").describe()

ingame_revenue                                                    \
                    count       mean        std    min    25%    50%    75%   
membership                                                                    
basic             98405.0   3.616504   8.370595   0.99   1.99   1.99   1.99   
plus              11722.0  28.157735  70.809548  12.91  14.98  14.98  17.98   

                    unit_sold                                                
                max     count      mean       std  min  25%  50%  75%   max  
membership                                                                   
basic        557.78   98405.0  1.383263  1.006746  1.0  1.0  1.0  1.0  42.0  
plus        1369.83   11722.0  2.621822  2.027129  1.0  2.0  2.0  3.0  48.0

In [18]:
cash.groupby(["membership", "product_name"])[["ingame_revenue", "unit_sold"]].sum().sort_values(["membership", "unit_sold"])

ingame_revenue  unit_sold
membership product_name                                        
basic      BATTLEGROUNDS PLUS                  25.98          2
           10000(+1200 Bonus) G-Coin         7699.47         77
           5000(+500 Bonus) G-Coin          13898.05        278
           2500(+200 Bonus) G-Coin          40935.73       1638
           1000(+50 Bonus) G-Coin           42632.67       4267
           500(+10 Bonus) G-Coin            30404.94       6092
           100 G-Coin                       25190.52      25433
           STARTER PACKAGE                 195094.67      98333
plus       5000(+500 Bonus) G-Coin          20046.51        401
           10000(+1200 Bonus) G-Coin        75992.68        760
           2500(+200 Bonus) G-Coin          32613.02       1305
           1000(+50 Bonus) G-Coin           17574.03       1759
           500(+10 Bonus) G-Coin            10430.50       2090
           100 G-Coin                        3914.82       3951
           STARTER PACKAGE                  17335.65       8747
           BATTLEGROUNDS PLUS              152157.76      11720

In [82]:
starter_buyer_cash_buy = pd.read_csv('./data/starter_buyer_cash_buy.csv')

In [83]:
starter_buy_time = starter_buyer_cash_buy[starter_buyer_cash_buy.product_id == "gcoinbundledesc.1016"][["account_id", "time"]]

In [84]:
starter_buyer_gcoin = mysql.read_table(spark, "labs", "starter_buyer_gcoin")

In [85]:
paid_gcoin_time = starter_buyer_gcoin.groupBy("account_id").agg(min("time").alias("buy_time")).toPandas()

In [88]:
paid_gcoin_time.shape

(76869, 2)

In [86]:
df = pd.merge(paid_gcoin_time, starter_buy_time, on="account_id")

In [92]:
df[["buy_time", "time"]] = df[["buy_time", "time"]].apply(pd.to_datetime)

In [93]:
df['time_diff'] = (df.buy_time - df.time).astype('timedelta64[h]')

In [79]:
df_pos = df_pos.groupby(["time_diff"], as_index=False)["account_id"].count()

In [80]:
df_pos["cum_sum"] = df_pos.account_id.cumsum()

In [73]:
df_pos.account_id.sum()


310

In [59]:
df_pos["percent"] = df_pos["cum_sum"]/18267

In [61]:
df_pos.to_csv("./data/time_diff.csv", index=False)

In [92]:
# basic & plus
user = load_data_mart("pc", "2022-01-12", "2022-02-08", "user_master").where((col("firstlogindate") >= "2022-01-12") & (col("lastlogindate") == col("date")))

In [11]:
# legacy
legacy_user = load_data_mart("pc", "2022-01-12", "2022-02-08", "user_master").where((col("firstlogindate") < "2022-01-12") & (col("lastlogindate") == col("date")))

In [93]:
starterpackage = load_data_mart("pc", "2022-01-12", "2022-02-08", "cash_mtx").where(col("product_id") == "gcoinbundledesc.1016")

In [31]:
legacy_user.withColumn("Nday", datediff(to_date("date"), to_date(lit("2022-01-12")))).select("date", "Nday").show(5, truncate=False)

+----------+----+
|date      |Nday|
+----------+----+
|2022-01-16|4   |
|2022-01-16|4   |
|2022-01-16|4   |
|2022-01-16|4   |
|2022-01-16|4   |
+----------+----+
only showing top 5 rows



In [62]:
legacy_first_login_after_f2p = legacy_user.groupBy("accountid").agg(min("date").alias("first_login"))

In [63]:
legacy_first_login_after_f2p.select(countDistinct('accountid')).show()

+-------------------------+
|count(DISTINCT accountid)|
+-------------------------+
|                  8162382|
+-------------------------+



In [64]:
legacy_user.select(countDistinct("accountid")).show()

+-------------------------+
|count(DISTINCT accountid)|
+-------------------------+
|                  8162382|
+-------------------------+



In [65]:
legacy_user_final = legacy_user.join(legacy_first_login_after_f2p, "accountid", "left")

In [73]:
datediff(legacy_user_final.date, legacy_user_final.first_login)

Column<b'datediff(date, first_login)'>

In [75]:
leg_user2 = legacy_user_final.withColumn("Nday", datediff("date", "first_login") + 1).withColumnRenamed("date", "logindate")
leg_starter2 = leg_user2.join(starterpackage, (leg_user2.accountid == starterpackage.account_id) & (leg_user2.logindate == starterpackage.date), "left")

In [76]:
leg_ndays2 = leg_starter2.groupBy("Nday").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("Nday", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("Nday").toPandas()
leg_ndays2.to_csv("./data/leg_ndays2.csv", index=False)

In [77]:
leg_ndays2

,Nday,au,pu,pur
0,1,8162382,702132,0.086020
1,2,3406182,130960,0.038448
2,3,2844086,71923,0.025289
3,4,2640809,53934,0.020423
4,5,2436045,33636,0.013808
5,6,2158214,25139,0.011648
6,7,2059105,21766,0.010571
7,8,2083484,21339,0.010242
8,9,1983907,18581,0.009366
9,10,1893555,17124,0.009043


In [94]:
# basic & plus
bp_user = user.withColumn("Nday", datediff(col("date"), col("firstlogindate")) + 1)
# legacy
# leg_user = legacy_user.withColumn("Nday", datediff(to_date("date"), to_date(lit("2022-01-12"))) + 1)

In [95]:
bp_user = bp_user.withColumnRenamed("date", "logindate")
# leg_user = leg_user.withColumnRenamed("date", "logindate")

In [96]:
bp_starter = bp_user.join(starterpackage, (bp_user.accountid == starterpackage.account_id) & (bp_user.logindate == starterpackage.date), "left")
# leg_starter = leg_user.join(starterpackage, (leg_user.accountid == starterpackage.account_id) & (leg_user.logindate == starterpackage.date), "left")

In [81]:
bp_daily_df = bp_starter.groupBy("logindate").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("logindate", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("logindate").toPandas()
# leg_daily_df = leg_starter.groupBy("logindate").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("logindate", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("logindate").toPandas()

In [82]:
bp_daily_df.to_csv("./data/bp_daily_df.csv", index=False)
# leg_daily_df.to_csv("./data/leg_daily_df.csv", index=False)

In [97]:
bp_ndays = bp_starter.groupBy("Nday").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("Nday", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("Nday").toPandas()
# leg_ndays = leg_starter.groupBy("Nday").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("Nday", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("Nday").toPandas()

In [98]:
bp_ndays.to_csv("./data/bp_ndays.csv", index=False)
# leg_ndays.to_csv("./data/leg_ndays.csv", index=False)

In [99]:
bp_ndays

,Nday,au,pu,pur
0,1,9623865,63446,0.006593
1,2,3433014,30182,0.008792
2,3,2400569,18441,0.007682
3,4,1907632,13457,0.007054
4,5,1550548,9808,0.006326
5,6,1297396,7932,0.006114
6,7,1128105,6718,0.005955
7,8,1096888,6091,0.005553
8,9,993220,5427,0.005464
9,10,912386,4920,0.005392


In [55]:
leg_ndays

,Nday,au,pu,pur
0,1,1956849,411644,0.210361
1,2,2051358,157637,0.076845
2,3,2111077,98831,0.046815
3,4,2479977,100999,0.040726
4,5,2459858,52984,0.021539
5,6,2019619,31803,0.015747
6,7,1938323,25419,0.013114
7,8,1858602,22272,0.011983
8,9,1878323,20283,0.010798
9,10,1950872,20369,0.010441


In [60]:
bp_user_list = bp_user.select("accountid").distinct().toPandas()["accountid"].tolist()

In [61]:
starterpackage.where(col("account_id").isin(bp_user_list)).select(count("*"), countDistinct("account_id")).show()

Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7f71f50e56a8>
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1293, in <lambda>
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 625, in _garbage_collect_object
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 920, in garbage_collect_object
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1145, in send_command
KeyboardInterrupt: 
Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7f71f7a59158>
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1293, in <lambda>
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 62

KeyboardInterrupt: 

In [165]:
bp_user = load_data_mart("pc", "2022-01-12", "2022-03-08", "user_master").where((col("firstlogindate") >= "2022-01-12") & (col("firstlogindate") <= "2022-02-08")  & (col("lastlogindate") == col("date")))
leg_user = load_data_mart("pc", "2022-01-12", "2022-03-08", "user_master").where((col("firstlogindate") < "2022-01-12")  & (col("lastlogindate") == col("date")))

In [166]:
leg_1st_login = leg_user.groupBy("accountid").agg(min("date").alias("first_login")).where(col("first_login") <= "2022-02-08")
leg_user = leg_user.join(leg_1st_login, "accountid")

In [167]:
bp_user = bp_user.withColumn("Nday", datediff("date", "firstlogindate") + 1).withColumnRenamed("date", "logindate")
leg_user = leg_user.withColumn("Nday", datediff("date", "first_login") + 1).withColumnRenamed("date", "logindate")

In [168]:
bp_user = bp_user.where(col("Nday") <= 28)
leg_user = leg_user.where(col("Nday") <= 28)

In [169]:
sp = load_data_mart("pc", "2022-01-12", "2022-03-08", "cash_mtx").where(col("product_id") == "gcoinbundledesc.1016")

In [170]:
bp_sp = bp_user.join(sp, (bp_user.accountid==sp.account_id) & (bp_user.logindate == sp.date), "left")
leg_sp = leg_user.join(sp, (leg_user.accountid==sp.account_id) & (leg_user.logindate == sp.date), "left")

In [215]:
day15 = leg_sp.where((col("Nday") >= 14) & (col("Nday") <= 15)).select("date", "time", "first_login", "Nday", "account_id").toPandas()

In [216]:
day15.to_csv("./data/day15.csv", index=False)

In [219]:
day15 = day15[~day15.account_id.isna()]

In [220]:
user_list = day15[(day15.Nday == 15)].account_id.unique().tolist()

In [221]:
len(user_list)

24324

In [222]:
day15[day15.Nday == 15].groupby("date", as_index=False)["account_id"].count().sort_values("account_id")

,date,account_id
26,2022-02-21,66
27,2022-02-22,74
19,2022-02-14,75
20,2022-02-15,77
21,2022-02-16,78
23,2022-02-18,83
22,2022-02-17,83
25,2022-02-20,97
24,2022-02-19,98
13,2022-02-08,135


# 나만의 상점 
PC: 1/26 ~ 2/16

In [198]:
gcoin = load_data_mart("pc", "2022-01-26", "2022-01-31", "gcoin_use")

In [202]:
yourshop = gcoin.where(col("event_name").like("%yourshop%"))

In [213]:
day15_yourshop =  yourshop.where(col("account_id").isin(day15[day15.Nday == 15].account_id.unique().tolist())).toPandas()

In [209]:
day15_yourshop.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     11974|
+--------------------------+



In [233]:
day15.columns = ["date", "sp_buy_time", "first_login_after_f2p", "Nday", "account_id"]
day15_yourshop = day15_yourshop[["date", "account_id", "product_id", "product_name", "free_use", "paid_use", "price_", "time"]]

In [234]:
day15[(day15.Nday == 15)].head()

,date,sp_buy_time,first_login_after_f2p,Nday,account_id
295,2022-01-26,2022-01-26T04:01:50.3012311Z,2022-01-12,15,account.0420118b0f944d79bf9539b882a940c0
397,2022-01-29,2022-01-29T20:38:31.8208429Z,2022-01-15,15,account.057ee308385b4b76af8f77e41851670e
626,2022-02-02,2022-02-02T11:37:59.9257953Z,2022-01-19,15,account.08edd76de20c4c7c89d4a3a26409de05
754,2022-01-26,2022-01-26T13:30:00.5135842Z,2022-01-12,15,account.0ab3aa7a287b473aaa76122f3d6b0ade
852,2022-01-27,2022-01-27T16:27:56.8582749Z,2022-01-13,15,account.0c1c6b5c07904e8b85e0ac09b91b1144


In [235]:
day15_yourshop.dtypes

date            object
account_id      object
product_id      object
product_name    object
free_use         int64
paid_use         int64
price_          object
time            object
dtype: object

In [241]:
# starter pakcage Nday 15에 구매한 유저들의 yourshop 구매
yourshop_sp_df = pd.merge(day15_yourshop, day15[day15.Nday == 15], on="account_id")

In [242]:
yourshop_sp_df.head()

,date_x,account_id,product_id,product_name,free_use,paid_use,price_,time,date_y,sp_buy_time,first_login_after_f2p,Nday
0,2022-01-26,account.01c2db907c4d42c0b1a850dad2dcc90f,itemdesc.11080061,Summer Lovin' Sunglasses,210,0,300.00,2022-01-26T14:03:30.082752Z,2022-01-26,2022-01-26T02:43:59.4867264Z,2022-01-12,15
1,2022-01-26,account.01c2db907c4d42c0b1a850dad2dcc90f,itemdesc.11050081,Rabbit Season Hoodie,200,0,500.00,2022-01-26T14:26:31.3048487Z,2022-01-26,2022-01-26T02:43:59.4867264Z,2022-01-12,15
2,2022-01-26,account.02cbb806337449baae77f46ae68d54a7,itemdesc.13000138,GOLDEN SHIBA CREW SET,470,0,1190.00,2022-01-26T12:01:58.882628Z,2022-01-26,2022-01-26T12:00:07.5325134Z,2022-01-12,15
3,2022-01-26,account.02cbb806337449baae77f46ae68d54a7,itemdesc.13000127,CULT CLASSICS WEAPON PACK,180,1000,2970.00,2022-01-26T12:01:47.1553916Z,2022-01-26,2022-01-26T12:00:07.5325134Z,2022-01-12,15
4,2022-01-26,account.1454e88887474483aa573863a2a0c009,itemdesc.13000033,CHEERLEADER SET,1310,0,1880.00,2022-01-26T08:03:43.8616196Z,2022-01-26,2022-01-26T08:13:10.1312343Z,2022-01-12,15


In [245]:
yourshop_sp_df[["date_x", "date_y"]] = yourshop_sp_df[["date_x", "date_y"]].astype(str)

In [246]:
yourshop_sp_df[(yourshop_sp_df["time"] > yourshop_sp_df["sp_buy_time"]) & (yourshop_sp_df.date_x == yourshop_sp_df.date_y)].account_id.nunique()

8504

In [250]:
24324-8504

15820

34.961355040289426

In [115]:
bp_ndays_df = bp_sp.groupBy("Nday").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("Nday", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("Nday").toPandas()
bp_ndays_df.to_csv("./data/bp_ndays_df.csv", index=False)

In [116]:
bp_ndays_df

,Nday,au,pu,pur
0,1,9623865,63446,0.006593
1,2,3471418,30636,0.008825
2,3,2452877,19034,0.007760
3,4,1972599,14064,0.007130
4,5,1626978,10499,0.006453
5,6,1382129,8608,0.006228
6,7,1221422,7405,0.006063
7,8,1197976,6739,0.005625
8,9,1093192,6055,0.005539
9,10,1013848,5533,0.005457


In [117]:
leg_ndays_df = leg_sp.groupBy("Nday").agg(countDistinct("accountid").alias("au"), countDistinct("account_id").alias("pu")).select("Nday", "au", "pu", (col("pu")/col("au")).alias("pur")).orderBy("Nday").toPandas()
leg_ndays_df.to_csv("./data/leg_ndays_df.csv", index=False)

In [118]:
leg_ndays_df

,Nday,au,pu,pur
0,1,8162382,702132,0.086020
1,2,3432241,131662,0.038360
2,3,2884543,72815,0.025243
3,4,2694170,54855,0.020361
4,5,2500952,34662,0.013860
5,6,2233957,26218,0.011736
6,7,2146958,22832,0.010635
7,8,2184914,22475,0.010286
8,9,2088398,19717,0.009441
9,10,2003166,18286,0.009129


In [123]:
bp_user = load_data_mart("pc", "2022-01-12", "2022-03-08", "user_master").where((col("firstlogindate") >= "2022-01-12") & (col("firstlogindate") <= "2022-02-08")  & (col("lastlogindate") == col("date")))
leg_user = load_data_mart("pc", "2022-01-12", "2022-03-08", "user_master").where((col("firstlogindate") < "2022-01-12")  & (col("lastlogindate") == col("date")))

In [124]:
leg_1st_login = leg_user.groupBy("accountid").agg(min("date").alias("first_login")).where(col("first_login") <= "2022-02-08")
leg_user = leg_user.join(leg_1st_login, "accountid")

In [125]:
bp_user = bp_user.withColumn("Nday", datediff("date", "firstlogindate") + 1).withColumnRenamed("date", "logindate").where(col("Nday") <= 28)
leg_user = leg_user.withColumn("Nday", datediff("date", "first_login") + 1).withColumnRenamed("date", "logindate").where(col("Nday") <= 28)

In [126]:
cash_item = load_data_mart("pc", "2022-01-12", "2022-03-08", "cash_mtx")

In [135]:
bp_cash = bp_user.join(cash_item, (bp_user.accountid==cash_item.account_id) & (bp_user.logindate == cash_item.date), "left")
leg_cash = leg_user.join(cash_item, (leg_user.accountid==cash_item.account_id) & (leg_user.logindate == cash_item.date), "left")

In [133]:
bp_nday_au = bp_user.groupBy("Nday").agg(countDistinct("accountid").alias("au"))
leg_nday_au = leg_user.groupBy("Nday").agg(countDistinct("accountid").alias("au"))

In [136]:
bp_nday_pu = bp_cash.groupBy("Nday", "product_name").agg(countDistinct("account_id").alias("pu"))
leg_nday_pu = leg_cash.groupBy("Nday", "product_name").agg(countDistinct("account_id").alias("pu"))

In [139]:
bp_nday_df = bp_nday_au.join(bp_nday_pu, "Nday").withColumn("pur", col("pu")/col("au"))
leg_nday_df = leg_nday_au.join(leg_nday_pu, "Nday").withColumn("pur", col("pu")/col("au"))

In [141]:
bp_nday_by_product = bp_nday_df.toPandas()
leg_nday_by_product = leg_nday_df.toPandas()

In [142]:
bp_nday_by_product.to_csv("./data/bp_nday_by_product.csv", index=False)
leg_nday_by_product.to_csv("./data/leg_nday_by_product.csv", index=False)

In [152]:
bp_nday_by_product[bp_nday_by_product.product_name != "GILT ELEGANCE - 1050 G-COIN SKIN PACK"].pivot_table(values="pur", index="Nday", columns=["product_name"])

product_name,100 G-Coin,1000(+50 Bonus) G-Coin,10000(+1200 Bonus) G-Coin,2500(+200 Bonus) G-Coin,500(+10 Bonus) G-Coin,5000(+500 Bonus) G-Coin,BATTLEGROUNDS PLUS,STARTER PACKAGE
Nday,,,,,,,,
1,0.000937,0.000525,0.000268,0.000320,0.000647,0.000055,0.002265,0.006593
2,0.002021,0.001019,0.000045,0.000548,0.001313,0.000093,0.002562,0.008825
3,0.002176,0.001055,0.000042,0.000508,0.001317,0.000099,0.002312,0.007760
4,0.002164,0.001040,0.000043,0.000529,0.001364,0.000103,0.002202,0.007130
5,0.002246,0.001108,0.000058,0.000554,0.001424,0.000114,0.002179,0.006453
6,0.002228,0.001144,0.000056,0.000594,0.001516,0.000116,0.002192,0.006228
7,0.002238,0.001163,0.000061,0.000635,0.001591,0.000154,0.002171,0.006063
8,0.002245,0.001183,0.000062,0.000637,0.001555,0.000125,0.002097,0.005625
9,0.002291,0.001231,0.000070,0.000606,0.001604,0.000135,0.002126,0.005539


In [147]:
bp_nday_by_product.groupby("Nday", as_index=False)["pur"].mean()

,Nday,pur
0,1,0.001161
1,2,0.001825
2,3,0.001697
3,4,0.001619
4,5,0.001571
5,6,0.001564
6,7,0.001564
7,8,0.001503
8,9,0.001511
9,10,0.001516


In [157]:
leg_nday_by_product[~leg_nday_by_product.product_name.isin(["None", "GILT ELEGANCE - 1050 G-COIN SKIN PACK"])].pivot_table(values="pur", index="Nday", columns=["product_name"])

product_name,100 G-Coin,1000(+50 Bonus) G-Coin,10000(+1200 Bonus) G-Coin,2500(+200 Bonus) G-Coin,500(+10 Bonus) G-Coin,5000(+500 Bonus) G-Coin,BATTLEGROUNDS PLUS,STARTER PACKAGE
Nday,,,,,,,,
1,0.009185,0.002909,0.000757,0.002295,0.004270,0.001005,0.000484,0.086020
2,0.009068,0.003220,0.000832,0.002537,0.003962,0.001081,0.000257,0.038360
3,0.007277,0.002872,0.000712,0.002255,0.003329,0.000926,0.000229,0.025243
4,0.006734,0.002832,0.000677,0.002190,0.003147,0.000906,0.000179,0.020361
5,0.006046,0.002619,0.000546,0.001948,0.002991,0.000756,0.000169,0.013860
6,0.004923,0.002499,0.000530,0.001767,0.002771,0.000669,0.000165,0.011736
7,0.004411,0.002456,0.000492,0.001740,0.002711,0.000673,0.000159,0.010635
8,0.004622,0.002529,0.000540,0.001813,0.002743,0.000694,0.000146,0.010286
9,0.004397,0.002531,0.000532,0.001791,0.002707,0.000708,0.000135,0.009441


In [154]:
leg_nday_by_product

,Nday,au,product_name,pu,pur
0,28,1362136,BATTLEGROUNDS PLUS,182,0.000134
1,28,1362136,100 G-Coin,6269,0.004602
2,28,1362136,None,0,0.000000
3,28,1362136,2500(+200 Bonus) G-Coin,2832,0.002079
4,28,1362136,1000(+50 Bonus) G-Coin,5299,0.003890
5,28,1362136,STARTER PACKAGE,7651,0.005617
6,28,1362136,500(+10 Bonus) G-Coin,5985,0.004394
7,28,1362136,5000(+500 Bonus) G-Coin,975,0.000716
8,28,1362136,10000(+1200 Bonus) G-Coin,655,0.000481
9,26,1383040,10000(+1200 Bonus) G-Coin,748,0.000541


In [144]:
leg_nday_by_product.groupby("Nday", as_index=False)["pur"].mean()

,Nday,pur
0,1,0.010693
1,2,0.006591
2,3,0.004284
3,4,0.004114
4,5,0.003215
5,6,0.002784
6,7,0.002586
7,8,0.002597
8,9,0.002471
9,10,0.002513
